## Importações

In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

## Carga de Dados

In [2]:
data_treino = pd.read_csv('/home/caio/github/k-3/data/train.csv')
data_teste = pd.read_csv('/home/caio/github/k-3/data/test.csv')
data_zero = pd.read_csv('/home/caio/github/k-3/data/test.csv')

## Feature Engineer

In [3]:
# Supondo que seu DataFrame se chame df
data_treino["cloud_humidity"] = data_treino["cloud"] * data_treino["humidity"]
data_treino["cloud_pressure"] = data_treino["cloud"] / data_treino["pressure"]
data_treino["cloud_windspeed"] = data_treino["cloud"] * data_treino["windspeed"]
data_treino["cloud_index"] = (data_treino["cloud"] + data_treino["humidity"]) / 2

data_treino["humidity_cloud"] = data_treino["humidity"] * data_treino["cloud"]
data_treino["humidity_pressure"] = data_treino["humidity"] / data_treino["pressure"]
data_treino["humidity_temp"] = data_treino["humidity"] * data_treino["temparature"]


In [4]:
# Supondo que seu DataFrame se chame df
data_teste["cloud_humidity"] = data_teste["cloud"] * data_teste["humidity"]
data_teste["cloud_pressure"] = data_teste["cloud"] / data_teste["pressure"]
data_teste["cloud_windspeed"] = data_teste["cloud"] * data_teste["windspeed"]
data_teste["cloud_index"] = (data_teste["cloud"] + data_teste["humidity"]) / 2

data_teste["humidity_cloud"] = data_teste["humidity"] * data_teste["cloud"]
data_teste["humidity_pressure"] = data_teste["humidity"] / data_teste["pressure"]
data_teste["humidity_temp"] = data_teste["humidity"] * data_teste["temparature"]

In [5]:
data_treino = data_treino.drop(columns=["dewpoint"])
data_treino = data_treino.drop(columns=["winddirection"])
data_treino = data_treino.drop(columns=["day"])
data_treino = data_treino.drop(columns=["id"])


In [6]:
data_teste = data_teste.drop(columns=["dewpoint"])
data_teste = data_teste.drop(columns=["day"])
data_teste = data_teste.drop(columns=["id"])
data_teste = data_teste.drop(columns=["winddirection"])

## Divisão Feature / Target

In [7]:
feature = data_treino.drop(columns=['rainfall'])       # Feature Treino
target = data_treino.pop('rainfall')                   # Target Treino
feature_final = data_teste                                    # Feature para Teste final

In [8]:
feature_treino, feature_teste, target_treino, target_teste = train_test_split(feature, target, test_size=0.2, random_state=69)

## Criação do Modelo

In [9]:
model = CatBoostClassifier(
    learning_rate=0.01, 
    l2_leaf_reg=10.0, 
    iterations=600, 
    depth=7, 
    border_count=80, 
    verbose=0, 
    random_state=69
)


## Treinamento do Modelo de Provisório

In [10]:
model.fit(feature_treino, target_treino)
previsao_verificadora = model.predict_proba(feature_teste)[:, 1]  # Pegando apenas a probabilidade da classe positiva

# Calculando AUC Score
auc_score = roc_auc_score(target_teste, previsao_verificadora)
print(f"AUC Score: {auc_score:.4f}")
# mvp 0.8704

AUC Score: 0.8686


## Modelo Final

In [11]:
model.fit(feature, target)
previsao = model.predict_proba(data_teste)[:, 1]  # Pega apenas a probabilidade da classe positiva
previsao = previsao.flatten()  # Ou previsao.ravel()

In [12]:
caminho_destino = 'ensemble/Cat_Classifier.csv'
resultado = pd.DataFrame({
    'id': data_zero['id'],
    'rainfall': previsao     
})

resultado.to_csv(caminho_destino, index=False)